In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
import matplotlib.pyplot as plt
import networkx as nx
import time
from evaluation import evaluate_all
from evaluation_functions import KFold

In [ ]:
data = KFold(cluster=True)

Set random seed and device as per the spec.

In [ ]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

The matrix Vecotrizer class is taken from https://github.com/basiralab/DGL/blob/main/Project/MatrixVectorizer.py to aid in vectorizing and anti-vectorizing our dataset. We develop functions later in the cell to do the same using an object of the class.

In [ ]:
class MatrixVectorizer:
    def _init_(self):
        pass

    @staticmethod
    def vectorize(matrix, include_diagonal=False):
        # Determine the size of the matrix based on its first dimension
        matrix_size = matrix.shape[0]
        # Initialize an empty list to accumulate vector elements
        vector_elements = []

        # Iterate over columns and then rows to collect the relevant elements
        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:  
                    if row < col:
                        # Collect upper triangle elements
                        vector_elements.append(matrix[row, col])
                    elif include_diagonal and row == col + 1:
                        # Optionally include the diagonal elements immediately below the diagonal
                        vector_elements.append(matrix[row, col])

        return np.array(vector_elements)

    @staticmethod
    def anti_vectorize(vector, matrix_size, include_diagonal=False):
        matrix = np.zeros((matrix_size, matrix_size))
        vector_idx = 0

        for col in range(matrix_size):
            for row in range(matrix_size):
                # Skip diagonal elements if not including them
                if row != col:  
                    if row < col:
                        # Reflect vector elements into the upper triangle and its mirror in the lower triangle
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1
                    elif include_diagonal and row == col + 1:
                        # Optionally fill the diagonal elements after completing each column
                        matrix[row, col] = vector[vector_idx]
                        matrix[col, row] = vector[vector_idx]
                        vector_idx += 1

        return matrix
    

mv = MatrixVectorizer()

def anti_vectorize_and_reshape(row, matrix_size):
    return mv.anti_vectorize(row, matrix_size)

def vectorise_and_reshape(matrix):
    return mv.vectorize(matrix)


Now we preprocess our data to antivectorize and convert it to a matrix form. We also clip and fillna as seen below.

In [ ]:
data.preprocessing()
print("Processing complete for train and test datasets.")

The idea behind symmetric normalization (using $D^{−1/2}AD^{−1/2}$, where D is the degree matrix and A is the adjacency matrix) is to weight the contribution of each node's features by the inverse square root of its degree, promoting balance in the influence of nodes regardless of their connectivity.

For a weighted adjacency matrix such as the one we have after antivectorising above, symmetric normalization applied below can be particularly useful when the scale of weights varies widely. This approach helps to mitigate the impact of very large or very small weights by normalizing the contributions of each edge in accordance with both the source and target nodes' weight. In essence, this normalization is useful because:

- It makes the scale of feature values and edge weights more uniform, which can be beneficial for learning algorithms that are sensitive to the scale of input data.
- The normalized weights can sometimes offer a more interpretable basis for understanding the relative importance of edges in the context of the entire graph structure.

In [ ]:
def normalize_adjacency_matrix(A):
    """
    Normalize the adjacency matrix with symmetric normalization.

    Args:
    A (numpy.ndarray): The adjacency matrix to normalize.

    Returns:
    numpy.ndarray: The normalized adjacency matrix.
    """
    # Add self-connections
    I = np.eye(A.shape[0])
    A_with_self_loops = A + I
    
    # Compute the degree matrix
    D_with_self_loops = np.array(np.sum(A_with_self_loops, axis=1)).reshape(-1)
    D_with_self_loops_inv_sqrt = np.power(D_with_self_loops, -0.5).flatten()
    D_with_self_loops_inv_sqrt[np.isinf(D_with_self_loops_inv_sqrt)] = 0.
    D_with_self_loops_inv_sqrt_matrix = np.diag(D_with_self_loops_inv_sqrt)
    
    # Compute the normalized adjacency matrix
    A_normalized = D_with_self_loops_inv_sqrt_matrix.dot(A_with_self_loops).dot(D_with_self_loops_inv_sqrt_matrix)
    
    return A_normalized

for i in range(3):
    data.lr[i] = data.lr[i].apply(lambda A: np.round(normalize_adjacency_matrix(A),4))

Some helper functions for our GSR, and the Graph U-Net class which is quite important for our GNN. This code is applied from https://github.com/basiralab/GSR-Net/tree/master. The Graph U-Net forward pass is described as follows:

- The input features X are first processed by an initial graph convolution layer. For A the adjacency matrix, X the input feature matrix, W the weight matrix, b​ bias term, and LeakyReLU the activation function, this can be represented by the equation:
$$X'= LeakyReLU(AXW​+b​)$$
- For each level i in self.ks, the features are further processed by a graph convolution layer, followed by a graph pooling operation. The pooling operation selects a subset of nodes based on top-k based on feature scores, reducing the graph size at each level.
- After downsampling to the smallest graph, the features are processed by self.bottom_gcn
- The process of UpSampling reverses the downsampling steps. Using stored indices from the pooling steps, graph unpooling operations reintegrate nodes back into the graph, followed by graph convolution layers to update the node features. The features from the corresponding downsampling level are added to the upsampled features to ensure the model captures both local and global graph structures.
- The upsampled features are concatenated with the original features processed by the initial graph convolution layer, and then passed through a final graph convolution layer to produce the output.

In [ ]:
def pad_HR_adj(label, split):

    label=np.pad(label,((split,split),(split,split)),mode="constant")
    np.fill_diagonal(label,1)
    return torch.from_numpy(label).type(torch.FloatTensor)

def unpad(data, split):
  
    idx_0 = data.shape[0]-split
    idx_1 = data.shape[1]-split
    train = data[split:idx_0, split:idx_1]
    return train

class GraphUnpool(nn.Module):

    def __init__(self):
        super(GraphUnpool, self).__init__()

    def forward(self, A, X, idx):
        new_X = torch.zeros([A.shape[0], X.shape[1]])
        new_X[idx] = X
        return A, new_X

    
class GraphPool(nn.Module):

    def __init__(self, k, in_dim):
        super(GraphPool, self).__init__()
        self.k = k
        self.proj = nn.Linear(in_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, A, X):
        scores = self.proj(X)
        # scores = torch.abs(scores)
        scores = torch.squeeze(scores)
        scores = self.sigmoid(scores/100)
        num_nodes = A.shape[0]
        values, idx = torch.topk(scores, int(self.k*num_nodes))
        new_X = X[idx, :]
        values = torch.unsqueeze(values, -1)
        new_X = torch.mul(new_X, values)
        A = A[idx, :]
        A = A[:, idx]
        return A, new_X, idx


class GCN(nn.Module):

    def __init__(self, in_dim, out_dim):
        super(GCN, self).__init__()
        self.proj = nn.Linear(in_dim, out_dim)
        self.drop = nn.Dropout(p=0.2)

    def forward(self, A, X):
        X = self.drop(X)
        X = self.proj(X)
        return X

class GraphUnet(nn.Module):

    def __init__(self, ks, in_dim, out_dim, dim=268):
        super(GraphUnet, self).__init__()
        self.ks = ks
       
        self.start_gcn = GCN(in_dim, dim)
        self.bottom_gcn = GCN(dim, dim)
        self.end_gcn = GCN(2*dim, out_dim)
        self.down_gcns = []
        self.up_gcns = []
        self.pools = []
        self.unpools = []
        self.l_n = len(ks)
        for i in range(self.l_n):
            self.down_gcns.append(GCN(dim, dim))
            self.up_gcns.append(GCN(dim, dim))
            self.pools.append(GraphPool(ks[i], dim))
            self.unpools.append(GraphUnpool())

    def forward(self, A, X):
        adj_ms = []
        indices_list = []
        down_outs = []
        X = self.start_gcn(A, X)
        start_gcn_outs = X
        org_X = X
        for i in range(self.l_n):
           
            X = self.down_gcns[i](A, X)
            adj_ms.append(A)
            down_outs.append(X)
            A, X, idx = self.pools[i](A, X)
            indices_list.append(idx)
        X = self.bottom_gcn(A, X)
        for i in range(self.l_n):
            up_idx = self.l_n - i - 1
           
            A, idx = adj_ms[up_idx], indices_list[up_idx]
            A, X = self.unpools[i](A, X, idx)
            X = self.up_gcns[i](A, X)
            X = X.add(down_outs[up_idx])
        X = torch.cat([X, org_X], 1)
        X = self.end_gcn(A, X)
        
        return X, start_gcn_outs

These are some more of the helper functions for our model class. Specifically, the interesting class is NodeAttention. The NodeAttention class defines a simple node-level attention mechanism for graph neural networks. This mechanism allows the model to weigh nodes differently based on their features, enabling it to focus more on important nodes during processing. The class is built with PyTorch and comprises two main parts: an attention scoring mechanism and the application of these scores to modulate the node features. The forward pass computes and applies attention as follows:

- It computes raw attention scores for each node by passing the node features through the linear layer. These raw scores are then normalized across nodes using the softmax function, ensuring that the scores sum up to 1 across the nodes for a given feature. This step can be represented by the equation, where W and b are the weights and bias of the linear layer self.attn, and x is the input feature matrix.:
$$attn_scores=softmax(Wx+b)$$
- The normalized attention scores are then element-wise multiplied with the original node features to yield the attended node features. This step modulates the importance of each node's features based on the learned attention scores.

The resulting attended feature matrix retains the original shape where each node's features are now scaled by its importance as determined by the model. This class effectively enables the network to focus on more relevant nodes by adjusting their influence on the model's output, which can be crucial for high resolution graph generation where not all nodes contribute equally to the final task.

In [ ]:

def normalize_adj_torch(mx):
    rowsum = mx.sum(1)
    r_inv_sqrt = torch.pow(rowsum, -0.5).flatten()
    r_inv_sqrt[torch.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = torch.diag(r_inv_sqrt)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    mx = torch.transpose(mx, 0, 1)
    mx = torch.matmul(mx, r_mat_inv_sqrt)
    return mx

def weight_variable_glorot(output_dim):

    input_dim = output_dim
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = np.random.uniform(-init_range, init_range,(input_dim, output_dim))
    
    return initial

class GSRLayer(nn.Module):
  
    def __init__(self,hr_dim):
        super(GSRLayer, self).__init__()

        self.weights = torch.from_numpy(weight_variable_glorot(hr_dim)).type(torch.FloatTensor)
        self.weights = torch.nn.Parameter(data=self.weights, requires_grad = True)
    def forward(self,A,X):
        lr = A
        lr_dim = lr.shape[0]
        f = X
        eig_val_lr, U_lr = torch.linalg.eigh(A, UPLO='U')
        eye_mat = torch.eye(lr_dim).type(torch.FloatTensor)
        s_d = torch.cat((eye_mat,eye_mat),0)
        s_d_adj = s_d[:268, :]
        a = torch.matmul(self.weights,s_d_adj )
        b = torch.matmul(a ,torch.t(U_lr))
        f_d = torch.matmul(b ,f)
        f_d = torch.abs(f_d)
        self.f_d = f_d.fill_diagonal_(1)
        adj = normalize_adj_torch(self.f_d)
        X = torch.mm(adj, adj.t())
        X = (X + X.t())/2
        idx = torch.eye(268, dtype=bool)
        X[idx]=1
        return adj, torch.abs(X)
    


class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, dropout=0.2, act=F.leaky_relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = torch.nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        input = F.dropout(input, self.dropout, self.training)
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        output = self.act(output)
        return output

class NodeAttention(nn.Module):
    def __init__(self, in_features):
        super(NodeAttention, self).__init__()
        self.in_features = in_features
        self.attn = nn.Linear(in_features, 1)
    def forward(self, x):
        # Compute attention scores
        attn_scores = F.softmax(self.attn(x), dim=1)
        # Apply attention scores
        attended_x = x * attn_scores
        return attended_x

Here, we have our model class, GSRapid. The name is relevant because out of all our model implementations this class trained the fastest due to its inherent simplicity. Our trials in AGSR and increasingly complex forward functions did not yield ideal results showing the relevance of the concept of Occam's Razor https://simple.wikipedia.org/wiki/Occam%27s_razor. Through synergising the existing GSR-Net model with NodeAttention, DropEdge, and relevant data augmentation in Symmetric Normalisation-- we were able to significantly improve the baseline performance. In this class, we introduce DropEdge:

DropEdge randomly removes a percentage of edges from the weight matrix of the graph in each training iteration. This operation is equivalent to applying dropout to the edges of the graph, which can be seen as a form of regularization. The goal is to prevent the model from over-relying on specific graph structures and to encourage the learning of more robust features that generalize well to unseen data. 

DropEdge is applied as follows in the class:

- Dropout is applied to the normalized weight matrix (A) using F.dropout(A, self.p_drop, training=self.training), where self.p_drop specifies the dropout rate. This operation randomly sets a fraction of elements in A to zero, effectively removing the edge weights and hence edges from the graph.

In the forward pass of GSRapid, we use the DropEdge-modified adjacency matrix to compute node representations at different scales. The process involves a Graph U-Net for hierarchical feature learning, followed by graph convolutions and NodeAttention to refine the node features. The final output is a symmetric matrix z, which has been regularized by DropEdge, promoting model generalization.

In [ ]:
class GSRapid(nn.Module):

    def __init__(self,ks,lr_dim=160,hr_dim=268,hidden_dim=268, p_drop=0.25):
        super(GSRapid, self).__init__()
        self.p_drop = p_drop
        self.lr_dim = lr_dim
        self.hr_dim = hr_dim
        self.hidden_dim = hidden_dim
        self.layer = GSRLayer(self.hr_dim)
        self.net = GraphUnet(ks, self.lr_dim, self.hr_dim)
        self.gc1 = GraphConvolution(self.hr_dim, self.hidden_dim, 0, act=F.leaky_relu)
        self.gc2 = GraphConvolution(self.hidden_dim, self.hr_dim, 0, act=F.leaky_relu)
        self.attn1 = NodeAttention(self.hidden_dim)
        self.attn2 = NodeAttention(self.hr_dim)

    def forward(self,lr):

        I = torch.eye(self.lr_dim).type(torch.FloatTensor)
        A = normalize_adj_torch(lr).type(torch.FloatTensor)
        A = F.dropout(A, self.p_drop, training=self.training)
        self.net_outs, self.start_gcn_outs = self.net(A, I)

        self.outputs, self.Z = self.layer(A, self.net_outs)

        self.hidden1 = self.gc1(self.Z, self.outputs)
        self.hidden1 = self.attn1(self.hidden1)
        self.hidden2 = self.gc2(self.hidden1, self.outputs)
        self.hidden2 = self.attn2(self.hidden2)

        z = self.hidden2
        z = (z + z.t())/2
        idx = torch.eye(self.hr_dim, dtype=bool) 
        z[idx]=1

        return torch.abs(z), self.net_outs, self.start_gcn_outs, self.outputs

Here, we have the train and test functions for GSRapid. Interestingly, we use MAE loss as the critereon instead of MSE. At first, theoretically we believed that using either would not change the model learning much as MSE is just a square function on MAE. But, MSE is more sensitive to outliers than MAE because the errors are squared before they are averaged, which disproportionately increases the influence of large errors. This means that models trained with MSE are likely to focus more on data points with larger errors, potentially at the expense of performing well on the majority of the data. This is what we observed as well. Switching to MAE yielded better performance on the public test set. 

In [ ]:
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj, subjects_labels, test_adj, test_labels, min_epochs=100, max_epochs=-1, early_stopping=10, padding=0, lmbda=32):

    best_model_dict = None
    best_loss = np.inf

    best_epoch = -1

    training_losses = []
    test_losses = []

    epochs_without_improvement = 0

    epoch = 0

    while (epochs_without_improvement < early_stopping or epoch < min_epochs) and (max_epochs == -1 or epoch < max_epochs):

        epoch+=1

        epoch_training_loss = []
        epoch_test_loss = []

        for lr,hr in zip(subjects_adj,subjects_labels):

            model.train()
            optimizer.zero_grad()

            lr = torch.from_numpy(lr).type(torch.FloatTensor)
            hr = torch.from_numpy(hr).type(torch.FloatTensor)

            model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
            model_outputs  = unpad(model_outputs, padding)
            padded_hr = pad_HR_adj(hr,padding)
            eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

            loss = lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr)
            epoch_training_loss.append(loss.item())

            loss.backward()
            optimizer.step()

        print("Epoch: ", epoch, "Training loss: ", np.mean(epoch_training_loss))
        training_losses.append(np.mean(epoch_training_loss))

        # Find test error if we want to early stop to seek the best number of epochs
        if max_epochs == -1:
            model.eval()

            for lr, hr in zip(test_adj,test_labels):

                lr = torch.from_numpy(lr).type(torch.FloatTensor)
                hr = torch.from_numpy(hr).type(torch.FloatTensor)

                model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
                model_outputs  = unpad(model_outputs, padding)
                padded_hr = pad_HR_adj(hr,padding)
                eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

                loss = lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr)
                epoch_test_loss.append(loss.item())

            epochs_without_improvement += 1

            print("Epoch: ", epoch, "Test loss: ", np.mean(epoch_test_loss))
            test_losses.append(np.mean(epoch_test_loss))

            if np.mean(epoch_test_loss) < best_loss:
                best_loss = np.mean(epoch_test_loss)
                best_model_dict = model.state_dict()
                epochs_without_improvement = 0
                best_epoch = epoch

    return (model, training_losses) if max_epochs != -1 else (best_epoch, training_losses, test_losses)
    

def test(model, test_adj, test_labels, file, padding=0):
    
    model.eval()

    all_preds = []

    for lr, hr in zip(test_adj,test_labels):

        lr = torch.from_numpy(lr).type(torch.FloatTensor)
        hr = torch.from_numpy(hr).type(torch.FloatTensor)

        preds,_,_,_ = model(lr)
        preds = unpad(preds, padding)
        all_preds.append(preds.detach().numpy())

    all_preds = np.array(all_preds)

    _test_labels = test_labels.to_numpy()
    _test_labels = np.array([np.array([np.array(x) for x in p]) for p in _test_labels])

    metrics = evaluate_all(
            _test_labels, all_preds, output_path=file
        )

    return all_preds, metrics

Here is the code for 3 fold cross validation. We track the memory usage as well as time taken to train and plot the memory usage at the end of the training. The testing takes significant time as processing different test metrics is very computationally heavy.

In [ ]:
# experimented with different values for ks
# ks = [0]
# ks = [0.9, 0.7, 0.6, 0.5, 0.4, 0.3]
ks = [0.9, 0.7, 0.6]

fold_results = []

def save_losses(test_index, training_losses, validation_losses=None):
    fig = plt.figure()
    plt.plot(training_losses, label='Training loss')
    if validation_losses is not None:
        plt.plot(validation_losses, label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Fold ' + str(test_index))
    plt.savefig(f'./loss_fold_{test_index}_validation.png' if validation_losses is not None else f'.loss_fold_{test_index}_training.png')
    plt.close(fig)

def find_max_epochs(test_index):
    model = GSRapid(ks)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    subjects_adj, subjects_ground_truth, test_adj, test_ground_truth = data.obtain_folds(test_index, with_validation=True)
    max_epochs, training_losses, validation_losses = train(model, optimizer, subjects_adj, subjects_ground_truth, test_adj, test_ground_truth)
    save_losses(test_index+1, training_losses, validation_losses)
    return max_epochs

total_training_time = 0
for test_index in range(3):
    start_time = time.time()
    max_epochs = find_max_epochs(test_index)
    print("Number of epochs for fold", test_index+1, "is", max_epochs)
    model = GSRapid(ks)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    subjects_adj, subjects_ground_truth, test_adj, test_ground_truth = data.obtain_folds(test_index, with_validation=False)
    model, losses = train(model, optimizer, subjects_adj, subjects_ground_truth, test_adj, test_ground_truth, max_epochs=max_epochs)
    total_training_time += time.time() - start_time
    save_losses(test_index+1, losses)
    predictions, metrics = test(model, test_adj, test_ground_truth, file=f'./Cluster CV/metrics.csv')
    fold_results.append(metrics)
    outputs = []
    for i,matrix in enumerate(predictions):
        vector = vectorise_and_reshape(matrix)
        outputs.append(vector)
    outputs_df = pd.DataFrame(outputs)
    meltedDF = outputs_df.to_numpy().flatten()
    predicted_df = pd.DataFrame({
    "Predicted": meltedDF
    })
    predicted_df['ID'] = range(1, len(meltedDF)+1)
    predicted_df = predicted_df[['ID', 'Predicted']]
    prediction_csv_path = f'./Cluster CV/predictions_fold_{test_index+1}.csv'
    predicted_df.to_csv(prediction_csv_path, index=False)
    
print(f"Total training time: {total_training_time} seconds.")